Nesse modelo utilizará a camada Embedding da Arquitetura BERT.

###Importando as bibliotecas

In [1]:
import numpy as np
import math
import re
import pandas as pd
from bs4 import BeautifulSoup
import random
from google.colab import drive

In [2]:
!pip install bert-for-tf2
!pip install sentencepiece

In [3]:
!pip install tensorflow==2.2.0-rc3

In [4]:
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow.keras import layers
import bert
tf.__version__
# !pip install tensorflow==2.2.0-rc3

'2.2.0-rc3'

###Carregamento dos Arquivos

In [5]:
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [6]:
cols = ['sentiment', 'id', 'date', 'query', 'user', 'text']

In [7]:
data = pd.read_csv('/content/drive/MyDrive/PLN/Classificação de Sentimento com BERT/training.1600000.processed.noemoticon.csv',
                   header = None,
                   names = cols,
                   engine='python',
                   encoding='latin1')

In [8]:
data.shape

(1600000, 6)

In [9]:
data.head()


,sentiment,id,date,query,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


###Pré-processamento

In [10]:
#Limpeza
def clean_tweet(tweet):
    tweet = BeautifulSoup(tweet, "lxml").get_text()
    tweet = re.sub(r"@[A-Za-z0-9]+", ' ', tweet)
    tweet = re.sub(r"https?://[A-Za-z0-9./]+", ' ', tweet)
    tweet = re.sub(r"[^a-zA-Z.!?']", ' ', tweet)
    tweet = re.sub(r" +", ' ', tweet)
    return tweet

In [11]:
data_clean = [clean_tweet(tweet) for tweet in data.text]

In [12]:
data_labels = data.sentiment.values
data_labels[data_labels == 4] = 1

### Tokenização

In [13]:
FullTokenizer = bert.bert_tokenization.FullTokenizer
bert_layer = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/1",
                            trainable=False)
vocab_file = bert_layer.resolved_object.vocab_file.asset_path.numpy()
do_lower_case = bert_layer.resolved_object.do_lower_case.numpy()
tokenizer = FullTokenizer(vocab_file, do_lower_case)

In [14]:
def encode_sentence(sent):
  return ["[CLS]"] + tokenizer.tokenize(sent) + ["[SEP]"]

In [15]:
encode_sentence("My dog likes strawberries.")

['[CLS]', 'my', 'dog', 'likes', 'straw', '##berries', '.', '[SEP]']

In [16]:
data_inputs = [encode_sentence(sentence) for sentence in data_clean]

In [17]:
print(data_inputs[0:2])

[['[CLS]', 'aw', '##w', '##w', 'that', "'", 's', 'a', 'bum', '##mer', '.', 'you', 'should', '##a', 'got', 'david', 'carr', 'of', 'third', 'day', 'to', 'do', 'it', '.', 'd', '[SEP]'], ['[CLS]', 'is', 'upset', 'that', 'he', 'can', "'", 't', 'update', 'his', 'facebook', 'by', 'text', '##ing', 'it', '.', '.', '.', 'and', 'might', 'cry', 'as', 'a', 'result', 'school', 'today', 'also', '.', 'blah', '!', '[SEP]']]


### Criação da base de dados

In [18]:
def get_ids(tokens):
  return tokenizer.convert_tokens_to_ids(tokens)

In [19]:
get_ids(tokenizer.tokenize("my cat like grapes."))

[2026, 4937, 2066, 16575, 1012]

In [20]:
def get_mask(tokens):
  return np.char.not_equal(tokens, "(PAD)").astype(int)

In [21]:
get_mask(tokenizer.tokenize("my cat like grapes."))

array([1, 1, 1, 1, 1])

In [22]:
def get_segments(tokens):
  seg_ids = []
  current_seg_id = 0
  for tok in tokens:
    seg_ids.append(current_seg_id)
    if tok == "[SEP]":
      current_seg_id = 1 - current_seg_id
  return seg_ids

In [23]:
print(data_inputs[0])

['[CLS]', 'aw', '##w', '##w', 'that', "'", 's', 'a', 'bum', '##mer', '.', 'you', 'should', '##a', 'got', 'david', 'carr', 'of', 'third', 'day', 'to', 'do', 'it', '.', 'd', '[SEP]']


In [24]:
get_segments(data_inputs[0])

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

In [25]:
my_sent = ["[CLS]"]+ tokenizer.tokenize("Roses are beautiful") +["[SEP]"]
my_sent

['[CLS]', 'roses', 'are', 'beautiful', '[SEP]']

In [26]:
bert_layer([
            tf.expand_dims(tf.cast(get_ids(my_sent), tf.int32),0),
            tf.expand_dims(tf.cast(get_mask(my_sent), tf.int32),0),
            tf.expand_dims(tf.cast(get_segments(my_sent), tf.int32),0)
          ])

[<tf.Tensor: shape=(1, 768), dtype=float32, numpy=
 array([[-9.01942849e-01, -3.59286964e-01, -4.81667012e-01,
          7.72407770e-01,  3.03145021e-01, -1.53753683e-01,
          9.09623086e-01,  2.33455285e-01, -3.11014801e-01,
         -9.99968350e-01, -2.53063262e-01,  7.49138534e-01,
          9.86961246e-01,  5.14592677e-02,  9.56657887e-01,
         -5.21562576e-01,  6.44484833e-02, -6.17905974e-01,
          3.85748565e-01, -6.78136349e-01,  7.31310010e-01,
          9.98348594e-01,  3.04465950e-01,  3.01394522e-01,
          4.86608267e-01,  9.08823371e-01, -6.43433690e-01,
          9.46487129e-01,  9.64284658e-01,  7.10817516e-01,
         -5.96172571e-01,  1.82842106e-01, -9.90260959e-01,
         -1.58804402e-01, -4.53707665e-01, -9.93388534e-01,
          3.19348425e-01, -7.84084797e-01,  4.81384061e-02,
         -4.91123693e-03, -9.08263087e-01,  3.38724315e-01,
          9.99753475e-01,  5.86365648e-02,  2.48249605e-01,
         -2.79796869e-01, -9.99998629e-01,  3.346

In [27]:
all_dataset = tf.data.Dataset.from_generator(lambda: sorted_all,
                                             output_types=(tf.int32, tf.int32))

In [28]:
BATCH_SIZE = 32
all_batched = all_dataset.padded_batch(BATCH_SIZE,
                                       padded_shapes = ((3, None), ()),
                                       padding_values=(0,0))

In [29]:
data_with_len = [[sent, data_labels[i], len(sent)]
                 for i, sent in enumerate(data_inputs)]

random.shuffle(data_with_len)
data_with_len.sort(key = lambda x: x[2])
sorted_all = [([get_ids(sent_lab[0]),
               get_mask(sent_lab[0]),
               get_segments(sent_lab[0])],

               sent_lab[1])
              for sent_lab in data_with_len if sent_lab[2]>7]

In [30]:
all_dataset = tf.data.Dataset.from_generator(lambda: sorted_all,
                                             output_types=(tf.int32, tf.int32))

In [31]:
all_batched = all_dataset.padded_batch(BATCH_SIZE,
                                       padded_shapes = ((3, None), ()),
                                       padding_values = (0,0))

In [32]:
NB_BATCHES = len(sorted_all) // BATCH_SIZE
NB_BATCHES_TEST = NB_BATCHES // 10
all_batched.shuffle(NB_BATCHES)
test_dataset = all_batched.take(NB_BATCHES_TEST)
train_dataset = all_batched.skip(NB_BATCHES_TEST)

###Construção do Modelo

In [48]:
class DCNNBERTEmbedding(tf.keras.Model):
    
    def __init__(self,
                 nb_filters=50,
                 FFN_units=512,
                 nb_classes=2,
                 dropout_rate=0.1,
                 name="dcnn"):
        super(DCNNBERTEmbedding, self).__init__(name=name)
        
        self.bert_layer = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/1",
                                         trainable = False)

        self.bigram = layers.Conv1D(filters=nb_filters,
                                    kernel_size=2,
                                    padding="valid",
                                    activation="relu")
        self.trigram = layers.Conv1D(filters=nb_filters,
                                     kernel_size=3,
                                     padding="valid",
                                     activation="relu")
        self.fourgram = layers.Conv1D(filters=nb_filters,
                                      kernel_size=4,
                                      padding="valid",
                                      activation="relu")
        self.pool = layers.GlobalMaxPool1D()
        self.dense_1 = layers.Dense(units=FFN_units, activation="relu")
        self.dropout = layers.Dropout(rate=dropout_rate)
        if nb_classes == 2:
            self.last_dense = layers.Dense(units=1,
                                           activation="sigmoid")
        else:
            self.last_dense = layers.Dense(units=nb_classes,
                                           activation="softmax")
    
    def embed_with_bert(self, all_tokens):
      _, embs = self.bert_layer([all_tokens[:, 0, :],
                                 all_tokens[:, 1, :],
                                 all_tokens[:, 2, :]])
      return embs

    def call(self, inputs, training):
        x = self.embed_with_bert(inputs)
        
        x_1 = self.bigram(x)
        x_1 = self.pool(x_1)
        x_2 = self.trigram(x)
        x_2 = self.pool(x_2)
        x_3 = self.fourgram(x)
        x_3 = self.pool(x_3)
        
        merged = tf.concat([x_1, x_2, x_3], axis=-1) # (batch_size, 3 * nb_filters)
        merged = self.dense_1(merged)
        merged = self.dropout(merged, training)
        output = self.last_dense(merged)
        
        return output

###Treinamento

In [49]:
NB_FILTERS = 100
FFN_UNITS = 256
NB_CLASSES = 2
DROPOUT_RATE = 0.2
BATCH_SIZE = 32
NB_EPOCHS = 5

In [50]:
Dcnn = DCNNBERTEmbedding(nb_filters=NB_FILTERS,
                         FFN_units=FFN_UNITS,
                         nb_classes=NB_CLASSES,
                         dropout_rate=DROPOUT_RATE)

In [51]:
if NB_CLASSES == 2:
    Dcnn.compile(loss='binary_crossentropy',
                 optimizer='adam', 
                 metrics=['accuracy']) #Usada quando temos 2 classes - no caso, positivo e negativo
else: 
    Dcnn.compile(loss='spare_categorical_crossentropy',
                 optmizer = 'adam',
                 metrixs =['sparse_categorical_accuracy'])

In [52]:
checkpoint_path = '/content/drive/MyDrive/PLN'

ckpt = tf.train.Checkpoint(Dcnn=Dcnn)

ckpt_manager = tf.train.CheckpointManager(ckpt, checkpoint_path, max_to_keep=1)

if ckpt_manager.latest_checkpoint:
  ckpt.restore(ckpt_manager.latest_checkpoint)
  print('Latest checkpoint restored!')

In [53]:
class MyCustomCallback(tf.keras.callbacks.Callback):

    def on_epoch_end(self, epoch, logs=None):
        ckpt_manager.save()
        print("Checkpoint saved at {}.".format(checkpoint_path))

**Acelerando o treinamento: Step_per_epoch = 100 é uma forma de definir um número para os registros. Porém, o resultado vai gerar uma Acurácia menor do que comparada quando não é definido o número de registros.**



In [55]:
history = Dcnn.fit(train_dataset,
                   epochs=NB_EPOCHS,
                   steps_per_epoch = 100,
                   callbacks=[MyCustomCallback()])

Epoch 1/5
100/100 [==============================] - 9s 94ms/step - loss: 0.3281 - accuracy: 0.8591
Epoch 2/5
100/100 [==============================] - 11s 110ms/step - loss: 0.3480 - accuracy: 0.8503
Epoch 3/5
100/100 [==============================] - 10s 102ms/step - loss: 0.3499 - accuracy: 0.8466
Epoch 4/5
100/100 [==============================] - 9s 92ms/step - loss: 0.3527 - accuracy: 0.8466
Epoch 5/5
100/100 [==============================] - 9s 91ms/step - loss: 0.3555 - accuracy: 0.8519


###Avaliação

In [56]:
results = Dcnn.evaluate(test_dataset)
print(results)

4513/4513 [==============================] - 266s 59ms/step - loss: 0.3503 - accuracy: 0.8494
[0.35025113821029663, 0.8493865132331848]


In [57]:
def get_prediction(sentence):
  tokens = encode_sentence(sentence)

  input_ids = get_ids(tokens)
  input_mask = get_mask(tokens)
  segment_ids = get_segments(tokens)

  inputs = tf.stack(
      [
       tf.cast(input_ids, dtype=tf.int32),
       tf.cast(input_mask, dtype=tf.int32),
       tf.cast(segment_ids, dtype=tf.int32),
      ], axis = 0)
  inputs = tf.expand_dims(inputs, 0)

  output = Dcnn(inputs, training=False)

  sentiment = math.floor(output*2)

  if sentiment == 0:
    print("Output of the model: {}\nPredicted sentiment: negative".format(output))
  elif sentiment == 1:
    print("Output of the model: {}\nPredicted sentiment: positive".format(output))


In [58]:
get_prediction("This actor is very bad.")

Output of the model: [[0.01695283]]
Predicted sentiment: negative


In [59]:
get_prediction("I like dogs.")

Output of the model: [[0.90014774]]
Predicted sentiment: positive
